#### Setup / imports

In [ ]:
# Demo version, e_RF_DT_org-dec.ipynb

# 2022-11-17 run F1 predictions on train-on-one
# building an outer loop to handle the multiple file reads per training cancer, 600
# Send to d_train-on-one plotting function

# run classical ML on both VAE input and decoded samples - 2022-11-15
# v17 plot series, start with self-decoded on across-cohort MAD, within-cohort column-wise normalization

# inputs from b_self.ipynb, a_GEXP_5k_col_norm.ipynb
    # Those code files output to:
    # g_devel/data_GEXP_5k_col_norm & g_devel/self-decoded_GEXP

# Original notes:
# output to d_F1_clf plot series, plot v16 for naming on the three-experiment audit
# retroactive / template note compilation versioning
# From e_RF_original.ipynb, re-run to fix gitignore error, 2022-11-11

In [ ]:
%whos

In [2]:
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import f1_score

from statistics import mean
from statistics import stdev

In [3]:
plt_v = 17 # Plot set version for file naming
re_samp = 5 # Resample at 100%, to train-test splits
trn_tst_splts = 5

### Original stacked F1 production for d_train-on-one experiment

### Across-cohort MAD, within-cohort column-wise normalization
Experiments 1 through 4:  
    Train and predict on same (self decode)
    Train-on-one, predict on another
    Step-up
    Trian-on-all

In [4]:
import glob

In [9]:
vae_inpt_pths = glob.glob('data_GEXP_5k_col_norm/*.tsv')

In [12]:
len(vae_inpt_pths) # Because LGGGBM dropped

25

In [ ]:
# sorted

In [15]:
vae_inpt_pths

['data_GEXP_5k_col_norm/ACC_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/SARC_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/BRCA_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/CESC_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/LIHCCHOL_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/KIRP_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/KIRCKICH_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/MESO_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/PCPG_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/HNSC_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/SKCM_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/PRAD_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/LUSC_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/THYM_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/LUAD_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/ESCC_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/UVM_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/BLCA_gexp_col_norm_5k.tsv',
 'data_GEXP_5k_col_norm/GEA_gexp_col_norm_5k.tsv',
 'data_

#### Current notes:

This e_ F1 scoring notebook has all experiments 
Other x_ file sets have each experiment in a separate notebook


original iteration strategy notes:  

This is for build-out on the self experiment (1) ASAP  
Forgo on-the-fly clean-up for demo  

Don't delete notes or previous code while working   
Save completed version as demo_devel then delete old code and development notes  
    Or as self_demo ? Goal is have rationally designed coded file names and still easily access template versions  
Then all the old notes and code will still be in here  
Use build train-on-one ( experiment 2 ) demo out of this   
This is the devel notebook for self_demo and train-on-one_demo ( eventually train-on-all#4 and step-up#3 )  

### F1 Classical ML RF DT on VAE input data
#### Experiment 0

F1 files written to g_devel/input-to-VAE_F1s; read from here into d_n overlay plot

In [37]:
RF_blnk_frm = pd.DataFrame()
DT_blnk_frm = pd.DataFrame()
for pth in vae_inpt_pths:
    print(pth)
    df = pd.read_csv(pth, sep="\t", index_col=0)
    c=df.index.name

    RF_F1_dct = {}
    DT_F1_dct = {}
    
    for j in list(range(0, re_samp)): # 5 resamplings, becomes each bar on plor
        X = df.iloc[:, 1:] # All the samples every loop
        y = df.iloc[:, 0]

        RF_output = [] # to append 5 trn/tst predictions then reset for next resampling
        DT_output = []

        for i in list(range(0, trn_tst_splts)): # 5 train-test splits, from setup
            X_train, X_test, y_train, y_test = train_test_split(
                X, y,
                    test_size=0.33,
                        )
            RF_clf = RandomForestClassifier(max_depth=2) # clf reset
            DT_clf = DecisionTreeClassifier()            

            RF_clf.fit(X_train, y_train)
            DT_clf.fit(X_train, y_train)

            RF_F1_scr = f1_score(y_test, RF_clf.predict(X_test), average='weighted')
            RF_output.append(RF_F1_scr)

            DT_F1_scr = f1_score(y_test, DT_clf.predict(X_test), average='weighted')
            DT_output.append(DT_F1_scr) # end tt loop

        RF_F1_dct['mean'+str(j)] = mean(RF_output) # Take the stats on the tt loop
        RF_F1_dct['std'+str(j)] = stdev(RF_output)

        DT_F1_dct['mean'+str(j)] = mean(DT_output)
        DT_F1_dct['std'+str(j)] = stdev(DT_output) # end sample loop, grown horzontally and ready to stack
    # break

    RF_chrt_frm = pd.DataFrame(RF_F1_dct, index = [c + '_trained_' +
                                             # c_not + '_predict_'+
                                             'self' + '_predict_'+                                                   
                                            'rndm_frst_clf'])
    
    # break
    RF_blnk_frm = pd.concat( [RF_blnk_frm, RF_chrt_frm] )
    
    


    DT_chrt_frm = pd.DataFrame(DT_F1_dct, index = [c + '_trained_' +
                                             # c_not + '_predict_'+
                                            'self' + '_predict_'+      
                                            'dcsn_tree_clf'])
    DT_blnk_frm = pd.concat( [DT_blnk_frm, DT_chrt_frm] )
    
    
    # manually moved here, fixe later, use for all four experients
    # /Users/karlberb/Documents/work/sample_count/synthetic_TCGA_data_gen/g_devel/input-to-VAE_F1s # <------------
    
RF_blnk_frm.to_csv(
        'self_F1s/RF_predict_on_self.tsv', # Use experiment # var for F1 output dir
        sep = '\t')
DT_blnk_frm.to_csv(
        'self_F1s/DT_predict_on_self.tsv', # Use experiment # var for F1 file name, n=24 / self / ..
        sep = '\t')
    
print('all done')    

data_GEXP_5k_col_norm/ACC_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/SARC_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/BRCA_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/CESC_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/LIHCCHOL_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/KIRP_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/KIRCKICH_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/MESO_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/PCPG_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/HNSC_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/SKCM_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/PRAD_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/LUSC_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/THYM_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/LUAD_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/ESCC_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/UVM_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/BLCA_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/GEA_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/THCA_gexp_col_norm_5k.tsv
data_GEXP_5k_col_norm/COADREAD_gexp

In [38]:
self_test = pd.read_csv('self_F1s/DT_predict_on_self.tsv', sep = '\t', index_col = 0)

In [39]:
self_test

,mean0,std0,mean1,std1,mean2,std2,mean3,std3,mean4,std4
ACC_trained_self_predict_dcsn_tree_clf,0.590999,0.146999,0.545774,0.082834,0.665867,0.072171,0.518553,0.093595,0.617137,0.088923
SARC_trained_self_predict_dcsn_tree_clf,0.865024,0.042474,0.882536,0.020766,0.862825,0.037138,0.869198,0.039125,0.879668,0.039719
BRCA_trained_self_predict_dcsn_tree_clf,0.808328,0.025744,0.829830,0.021523,0.812698,0.014012,0.799509,0.024133,0.827105,0.020962
CESC_trained_self_predict_dcsn_tree_clf,0.743142,0.057241,0.734853,0.053452,0.726054,0.092308,0.755560,0.056928,0.760765,0.032713
LIHCCHOL_trained_self_predict_dcsn_tree_clf,0.635467,0.031217,0.651368,0.034239,0.618614,0.032443,0.627112,0.029720,0.684142,0.044030
KIRP_trained_self_predict_dcsn_tree_clf,0.695160,0.023397,0.741861,0.064832,0.685497,0.033494,0.701543,0.056339,0.716048,0.035486
KIRCKICH_trained_self_predict_dcsn_tree_clf,0.951764,0.011096,0.954792,0.007475,0.944910,0.026896,0.960523,0.012233,0.967342,0.009790
MESO_trained_self_predict_dcsn_tree_clf,0.454055,0.150491,0.556269,0.052467,0.532335,0.072419,0.489929,0.058594,0.558088,0.044612
PCPG_trained_self_predict_dcsn_tree_clf,0.729581,0.062906,0.696561,0.051281,0.739648,0.076552,0.751747,0.053451,0.709626,0.040178
HNSC_trained_self_predict_dcsn_tree_clf,0.572405,0.059667,0.588782,0.031505,0.643661,0.040249,0.577748,0.022781,0.612300,0.040231


In [ ]:
# 2022-11-15, Tuesday, generate F1s for Self Decode Experiment using input to VAE experiment

# Used as Template for above
RF_blnk_frm = pd.DataFrame()
DT_blnk_frm = pd.DataFrame()

# for c in org.Labels.str.split('_').str[0].unique()[:2]: # use c for cohort

for c in clist: # use c for cohort
    print(c)
    # primary_subset = org[org.Labels.str.contains(c)]
    RF_blnk_frm = pd.DataFrame()
    DT_blnk_frm = pd.DataFrame()
    # re_samp = re_samp
    # trn_tst_splts = trn_tst_splts
    for c_path in [s for s in trn_on_one_paths if s.split('/')[1].startswith(c)]:
        print(c_path)
        c_not = c_path.split('_')[3]
        # df = pd.read_csv(c_path, sep="\t", index_col=0)
        # break
        RF_F1_dct = {} # to a row to stack into the blank frame, refresh at the start of each cancer row
        DT_F1_dct = {}

        # break
        for j in list(range(0, re_samp)): # step through five resamplings
            X = df.iloc[:, 1:] # take all the samples in each of five times through the loop
            y = df.iloc[:, 0]

            RF_output = [] # tack on five tt predictions then reset for next resampling
            DT_output = []

            for i in list(range(0, trn_tst_splts)): # five train-test splits
                X_train, X_test, y_train, y_test = train_test_split(
                    X, y,
                        test_size=0.33,
                            )
                RF_clf = RandomForestClassifier(max_depth=2) # intitialize a fresh clf each tt splt
                DT_clf = DecisionTreeClassifier()            

                RF_clf.fit(X_train, y_train)
                DT_clf.fit(X_train, y_train)

                RF_F1_scr = f1_score(y_test, RF_clf.predict(X_test), average='weighted')
                RF_output.append(RF_F1_scr)

                DT_F1_scr = f1_score(y_test, DT_clf.predict(X_test), average='weighted')
                DT_output.append(DT_F1_scr) # end tt loop

            RF_F1_dct['mean'+str(j)] = mean(RF_output) # Take the stats on the tt loop
            RF_F1_dct['std'+str(j)] = stdev(RF_output)

            DT_F1_dct['mean'+str(j)] = mean(DT_output)
            DT_F1_dct['std'+str(j)] = stdev(DT_output) # end sample loop, grown horzontally and ready to stack

            
        RF_chrt_frm = pd.DataFrame(RF_F1_dct, index = [c + '_trained_' +
                                                 c_not + '_predict_'+
                                                'rndm_frst_clf'])
        RF_blnk_frm = pd.concat( [RF_blnk_frm, RF_chrt_frm] )            

        
        DT_chrt_frm = pd.DataFrame(DT_F1_dct, index = [c + '_trained_' +
                                                 c_not + '_predict_'+
                                                'dcsn_tree_clf'])
        DT_blnk_frm = pd.concat( [DT_blnk_frm, DT_chrt_frm] )
        
        # break # c    
    # break    
    RF_blnk_frm.to_csv(
            'train-on-one_F1s/'+c+'_RF_predict_on_n=24.tsv',
            sep = '\t')
    DT_blnk_frm.to_csv(
            'train-on-one_F1s/'+c+'_DT_predict_on_n=24.tsv',
            sep = '\t')

print('all done') # still no auto-file writing, interactive mode

In [ ]:
# End classial F1s on VAE input

### Self-decoded F1s
#### Experiment 1

F1 score file written to self_F1s, read into d_self overlay 

In [94]:
self_decoded_pths = sorted(glob.glob('self-decoded_GEXP/*.tsv'))

In [95]:
self_decoded_pths

['self-decoded_GEXP/ACC_GEXP_self_decoded_3.91_fit_time.tsv',
 'self-decoded_GEXP/BLCA_GEXP_self_decoded_11.66_fit_time.tsv',
 'self-decoded_GEXP/BRCA_GEXP_self_decoded_27.66_fit_time.tsv',
 'self-decoded_GEXP/CESC_GEXP_self_decoded_6.86_fit_time.tsv',
 'self-decoded_GEXP/COADREAD_GEXP_self_decoded_13.69_fit_time.tsv',
 'self-decoded_GEXP/ESCC_GEXP_self_decoded_4.55_fit_time.tsv',
 'self-decoded_GEXP/GEA_GEXP_self_decoded_14.49_fit_time.tsv',
 'self-decoded_GEXP/HNSC_GEXP_self_decoded_15.62_fit_time.tsv',
 'self-decoded_GEXP/KIRCKICH_GEXP_self_decoded_17.26_fit_time.tsv',
 'self-decoded_GEXP/KIRP_GEXP_self_decoded_6.8_fit_time.tsv',
 'self-decoded_GEXP/LIHCCHOL_GEXP_self_decoded_12.38_fit_time.tsv',
 'self-decoded_GEXP/LUAD_GEXP_self_decoded_14.25_fit_time.tsv',
 'self-decoded_GEXP/LUSC_GEXP_self_decoded_14.98_fit_time.tsv',
 'self-decoded_GEXP/MESO_GEXP_self_decoded_4.96_fit_time.tsv',
 'self-decoded_GEXP/OV_GEXP_self_decoded_10.25_fit_time.tsv',
 'self-decoded_GEXP/PAAD_GEXP_self_dec

In [96]:
# F1 scores on self-decoding experiments

RF_blnk_frm = pd.DataFrame()
DT_blnk_frm = pd.DataFrame()

# for pth in self_decoded_pths: # Toggle here for vae-in / x-decoded
for pth in self_decoded_pths: # Toggle here for train-on-one_decoded
        
    print(pth)
    df = pd.read_csv(pth, sep="\t", index_col=0)
    c=df.index.name

    RF_F1_dct = {}
    DT_F1_dct = {}
    
    for j in list(range(0, re_samp)): # 5 resamplings, becomes each bar on plor
        X = df.iloc[:, 1:] # All the samples every loop
        y = df.iloc[:, 0]

        RF_output = [] # to append 5 trn/tst predictions then reset for next resampling
        DT_output = []

        for i in list(range(0, trn_tst_splts)): # 5 train-test splits, from setup
            X_train, X_test, y_train, y_test = train_test_split(
                X, y,
                    test_size=0.33,
                        )
            RF_clf = RandomForestClassifier(max_depth=2) # clf reset
            DT_clf = DecisionTreeClassifier()            

            RF_clf.fit(X_train, y_train)
            DT_clf.fit(X_train, y_train)

            RF_F1_scr = f1_score(y_test, RF_clf.predict(X_test), average='weighted')
            RF_output.append(RF_F1_scr)

            DT_F1_scr = f1_score(y_test, DT_clf.predict(X_test), average='weighted')
            DT_output.append(DT_F1_scr) # end tt loop

        RF_F1_dct['mean'+str(j)] = mean(RF_output) # Take the stats on the tt loop
        RF_F1_dct['std'+str(j)] = stdev(RF_output)

        DT_F1_dct['mean'+str(j)] = mean(DT_output)
        DT_F1_dct['std'+str(j)] = stdev(DT_output) # end sample loop, grown horzontally and ready to stack
    # break

    RF_chrt_frm = pd.DataFrame(RF_F1_dct, index = [c + '_trained_' +
                                             # c_not + '_predict_'+
                                             'self' + '_predict_'+                                                   
                                            'rndm_frst_clf'])
    
    # break
    RF_blnk_frm = pd.concat( [RF_blnk_frm, RF_chrt_frm] )
    
    


    DT_chrt_frm = pd.DataFrame(DT_F1_dct, index = [c + '_trained_' +
                                             # c_not + '_predict_'+ # Step up
                                            'self' + '_predict_'+      
                                            'dcsn_tree_clf'])
    DT_blnk_frm = pd.concat( [DT_blnk_frm, DT_chrt_frm] )
    
    
    # manually moved here, fixe later, use for all four experients
    # /Users/karlberb/Documents/work/sample_count/synthetic_TCGA_data_gen/g_devel/input-to-VAE_F1s
    
RF_blnk_frm.to_csv(
        'self_F1s/RF_self-decoded_F1s.tsv', # Use experiment # var for F1 output dir
        sep = '\t')
DT_blnk_frm.to_csv(
        'self_F1s/DT_self-decoded_F1s.tsv', # Use experiment # var for F1 file name, n=24 / self / ..
        sep = '\t')
    
print('all done')

self-decoded_GEXP/ACC_GEXP_self_decoded_3.91_fit_time.tsv
self-decoded_GEXP/BLCA_GEXP_self_decoded_11.66_fit_time.tsv
self-decoded_GEXP/BRCA_GEXP_self_decoded_27.66_fit_time.tsv
self-decoded_GEXP/CESC_GEXP_self_decoded_6.86_fit_time.tsv
self-decoded_GEXP/COADREAD_GEXP_self_decoded_13.69_fit_time.tsv
self-decoded_GEXP/ESCC_GEXP_self_decoded_4.55_fit_time.tsv
self-decoded_GEXP/GEA_GEXP_self_decoded_14.49_fit_time.tsv
self-decoded_GEXP/HNSC_GEXP_self_decoded_15.62_fit_time.tsv
self-decoded_GEXP/KIRCKICH_GEXP_self_decoded_17.26_fit_time.tsv
self-decoded_GEXP/KIRP_GEXP_self_decoded_6.8_fit_time.tsv
self-decoded_GEXP/LIHCCHOL_GEXP_self_decoded_12.38_fit_time.tsv
self-decoded_GEXP/LUAD_GEXP_self_decoded_14.25_fit_time.tsv
self-decoded_GEXP/LUSC_GEXP_self_decoded_14.98_fit_time.tsv
self-decoded_GEXP/MESO_GEXP_self_decoded_4.96_fit_time.tsv
self-decoded_GEXP/OV_GEXP_self_decoded_10.25_fit_time.tsv
self-decoded_GEXP/PAAD_GEXP_self_decoded_5.66_fit_time.tsv
self-decoded_GEXP/PCPG_GEXP_self_decode

In [ ]:
# Fix and check the file path naming

In [ ]:
self_dec_test = pd.read_csv('self_F1s/DT_predict_on_self.tsv', sep = '\t', index_col = 0)

In [ ]:
#### End decode-on-self 2022-11-15

### Train-on-one, decode on another F1 scoring
#### Experiment 2

F1 scores written to: g_devel/train-on-one_F1s

In [54]:
dec_n = pd.read_csv(
    train_on_one_decoded_pths[0],
                    sep = '\t', index_col = 0)

In [70]:
dec_n

,Labels,N:GEXP::ALB:213:,N:GEXP::TG:7038:,N:GEXP::COL1A1:1277:,N:GEXP::FN1:2335:,N:GEXP::KRT5:3852:,N:GEXP::KRT14:3861:,N:GEXP::KRT6A:3853:,N:GEXP::ADAM6:8755:,N:GEXP::SERPINA1:5265:,...,N:GEXP::C14orf1:11161:,N:GEXP::SMCHD1:23347:,N:GEXP::ARL2BP:23568:,N:GEXP::PITPNB:23760:,N:GEXP::KDM5A:5927:,N:GEXP::CNIH:10175:,N:GEXP::ACADS:35:,N:GEXP::YIPF2:78992:,N:GEXP::MEPCE:56257:,N:GEXP::PPARG:5468:
BLCA,,,,,,,,,,,,,,,,,,,,,
TCGA-2F-A9KO,BLCA_2,0.254426,0.321170,0.290030,0.265205,0.273064,0.284997,0.251193,0.279805,0.272279,...,0.378975,0.570919,0.441249,0.486293,0.492014,0.478890,0.406867,0.441323,0.489163,0.341754
TCGA-2F-A9KP,BLCA_2,0.189841,0.273588,0.227529,0.212832,0.219843,0.235837,0.191818,0.217314,0.187231,...,0.400651,0.571046,0.440698,0.501254,0.494149,0.475617,0.405889,0.437859,0.475112,0.291778
TCGA-2F-A9KQ,BLCA_2,0.252426,0.325898,0.277382,0.263099,0.276613,0.285857,0.242901,0.277120,0.245382,...,0.391922,0.543434,0.474944,0.494553,0.481174,0.481053,0.437613,0.462324,0.473476,0.333178
TCGA-2F-A9KR,BLCA_1,0.256926,0.349632,0.296242,0.278037,0.284370,0.286165,0.265202,0.284413,0.280915,...,0.386069,0.542955,0.455856,0.475517,0.469776,0.475787,0.406058,0.453049,0.479999,0.348216
TCGA-2F-A9KT,BLCA_1,0.233347,0.324335,0.273130,0.250021,0.244851,0.253598,0.241791,0.252571,0.247384,...,0.392416,0.562376,0.448199,0.522415,0.453464,0.477013,0.378085,0.439109,0.482926,0.310867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-ZF-AA56,BLCA_1,0.271012,0.348174,0.313581,0.312733,0.310199,0.307472,0.286383,0.306646,0.298388,...,0.416745,0.558791,0.450011,0.513456,0.472410,0.509470,0.428313,0.443667,0.491418,0.365635
TCGA-ZF-AA58,BLCA_1,0.243670,0.311282,0.292996,0.302640,0.284752,0.245034,0.276873,0.259288,0.283653,...,0.393148,0.560955,0.459743,0.507224,0.431732,0.462904,0.364904,0.444028,0.491883,0.357058
TCGA-ZF-AA5H,BLCA_1,0.234566,0.326497,0.283094,0.308321,0.281766,0.257971,0.271097,0.265458,0.274116,...,0.397449,0.566342,0.447698,0.518038,0.466147,0.507879,0.376257,0.449396,0.480247,0.337271


In [75]:
len(train_on_one_decoded_pths) # 600 decoded gexp files

600

In [63]:
train_on_one_decoded_pths = sorted(glob.glob('train-on-one_decoded/*.tsv')) # set up for the burley nested loops
cohorts = []
for path in train_on_one_decoded_pths:
    # print(path)
    # c=path.split
    cohorts.append(path.split('/')[1].split('_')[0])
clist = sorted(list(set(cohorts)))

In [64]:
clist

['ACC',
 'BLCA',
 'BRCA',
 'CESC',
 'COADREAD',
 'ESCC',
 'GEA',
 'HNSC',
 'KIRCKICH',
 'KIRP',
 'LIHCCHOL',
 'LUAD',
 'LUSC',
 'MESO',
 'OV',
 'PAAD',
 'PCPG',
 'PRAD',
 'SARC',
 'SKCM',
 'TGCT',
 'THCA',
 'THYM',
 'UCEC',
 'UVM']

In [65]:
len(clist) # Full set minus LGGGBM

25

In [67]:
c # cancer cohort

'ACC'

In [ ]:
##### Make a subset list of all the cancers decoded by the VAE trained on this training cancer, c

In [68]:
[s for s in train_on_one_decoded_pths if s.split('/')[1].startswith(c)]

['train-on-one_decoded/ACC_trained_BLCA_decoded_3.43_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_BRCA_decoded_3.43_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_CESC_decoded_3.43_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_COADREAD_decoded_3.43_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_ESCC_decoded_3.43_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_GEA_decoded_3.43_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_HNSC_decoded_3.43_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_KIRCKICH_decoded_3.43_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_KIRP_decoded_3.43_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_LIHCCHOL_decoded_3.43_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_LUAD_decoded_3.43_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_LUSC_decoded_3.43_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_MESO_decoded_3.43_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_OV_decoded_3.43_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_PAAD

In [87]:
# F1 scores on Train-on-one experiment, 2022-11-17 Thursday morning, Experiment #2
# Training cancer loop inserted from self-decode template above

# RF_blnk_frm = pd.DataFrame()
# DT_blnk_frm = pd.DataFrame()

for c in clist: # Cancer used for training
    RF_blnk_frm = pd.DataFrame()
    DT_blnk_frm = pd.DataFrame()
    print(c, 'is the train cancer')
    # break                                           # 24 cancers that were predicted
    for c_path in [s for s in train_on_one_decoded_pths if s.split('/')[1].startswith(c)]:

        c_not = c_path.split('_')[3]
        print(c_not, 'is the decode cancer')
        df = pd.read_csv(c_path, sep="\t", index_col=0) ### re-run
        # break
        RF_F1_dct = {} # to a row to stack into the blank frame, refresh at the start of each cancer row
        DT_F1_dct = {}        
        # break

        for j in list(range(0, re_samp)): # 5 resamplings, becomes each bar on plor
            print('resampling: ', j)
            X = df.iloc[:, 1:] # All the samples every loop
            y = df.iloc[:, 0]
            # break
            RF_output = [] # to append 5 trn/tst predictions then reset for next resampling
            DT_output = []

            for i in list(range(0, trn_tst_splts)): # 5 train-test splits, from setup
                # print(i, ' is i')
                X_train, X_test, y_train, y_test = train_test_split(
                    X, y,
                        test_size=0.33,
                            )
                RF_clf = RandomForestClassifier(max_depth=2) # clf reset
                DT_clf = DecisionTreeClassifier()            

                RF_clf.fit(X_train, y_train)
                DT_clf.fit(X_train, y_train)

                RF_F1_scr = f1_score(y_test, RF_clf.predict(X_test), average='weighted')
                RF_output.append(RF_F1_scr)

                DT_F1_scr = f1_score(y_test, DT_clf.predict(X_test), average='weighted')
                DT_output.append(DT_F1_scr) # end train-test repeat loop

            RF_F1_dct['mean'+str(j)] = mean(RF_output) # get error bar stats for overlay plot
            RF_F1_dct['std'+str(j)] = stdev(RF_output)

            DT_F1_dct['mean'+str(j)] = mean(DT_output)
            DT_F1_dct['std'+str(j)] = stdev(DT_output)
        # end the resampling loop 

        RF_chrt_frm = pd.DataFrame(RF_F1_dct, index = [c + '_trained_' +
                                                 c_not + '_predict_'+  # file naming train-on-one experiment                                             
                                                'rndm_frst_clf'])
        RF_blnk_frm = pd.concat( [RF_blnk_frm, RF_chrt_frm] )

        DT_chrt_frm = pd.DataFrame(DT_F1_dct, index = [c + '_trained_' +
                                                 c_not + '_predict_'+
                                                'dcsn_tree_clf'])
        DT_blnk_frm = pd.concat( [DT_blnk_frm, DT_chrt_frm] )
    # end the training cancer loop
    # break
    RF_blnk_frm.to_csv(
            'train-on-one_F1s/'+c+'_RF_predict_on_n=24.tsv', # Use experiment # var for F1 output dir
            sep = '\t')
    DT_blnk_frm.to_csv(
            'train-on-one_F1s/'+c+'_DT_predict_on_n=24.tsv', # Use experiment # var for F1 file name, n=24 / self / ..
            sep = '\t')
    print('cohort done')   
    # break
print('all done')

ACC  is the train cancer
BLCA  is the decode cancer
resampling:  0
resampling:  1
resampling:  2
resampling:  3
resampling:  4
BRCA  is the decode cancer
resampling:  0
resampling:  1
resampling:  2
resampling:  3
resampling:  4
CESC  is the decode cancer
resampling:  0
resampling:  1
resampling:  2
resampling:  3
resampling:  4
COADREAD  is the decode cancer
resampling:  0
resampling:  1
resampling:  2
resampling:  3
resampling:  4
ESCC  is the decode cancer
resampling:  0
resampling:  1
resampling:  2
resampling:  3
resampling:  4
GEA  is the decode cancer
resampling:  0
resampling:  1
resampling:  2
resampling:  3
resampling:  4
HNSC  is the decode cancer
resampling:  0
resampling:  1
resampling:  2
resampling:  3
resampling:  4
KIRCKICH  is the decode cancer
resampling:  0
resampling:  1
resampling:  2
resampling:  3
resampling:  4
KIRP  is the decode cancer
resampling:  0
resampling:  1
resampling:  2
resampling:  3
resampling:  4
LIHCCHOL  is the decode cancer
resampling:  0
resa

In [ ]:
### End train-on-one component of plot series 17

### Step-up F1 scoring
#### Experiment 3

F1 scores written to: g_devel/train-on-one_F1s

### Train-on-all-but-one F1 scoring
#### Experiment 4

F1 scores written to: g_devel/train-on-one_F1s

In [102]:
trn_on_all_pths = sorted(glob.glob('train-on-all_decoded/*.tsv'))

In [103]:
trn_on_all_pths

['train-on-all_decoded/trn-on-n24_ACC_decoded,75.71_fit_time.tsv',
 'train-on-all_decoded/trn-on-n24_BLCA_decoded,70.71_fit_time.tsv',
 'train-on-all_decoded/trn-on-n24_BRCA_decoded,67.79_fit_time.tsv',
 'train-on-all_decoded/trn-on-n24_CESC_decoded,75.7_fit_time.tsv',
 'train-on-all_decoded/trn-on-n24_COADREAD_decoded,73.74_fit_time.tsv',
 'train-on-all_decoded/trn-on-n24_ESCC_decoded,75.57_fit_time.tsv',
 'train-on-all_decoded/trn-on-n24_GEA_decoded,73.83_fit_time.tsv',
 'train-on-all_decoded/trn-on-n24_HNSC_decoded,74.98_fit_time.tsv',
 'train-on-all_decoded/trn-on-n24_KIRCKICH_decoded,74.78_fit_time.tsv',
 'train-on-all_decoded/trn-on-n24_KIRP_decoded,78.49_fit_time.tsv',
 'train-on-all_decoded/trn-on-n24_LIHCCHOL_decoded,78.19_fit_time.tsv',
 'train-on-all_decoded/trn-on-n24_LUAD_decoded,75.52_fit_time.tsv',
 'train-on-all_decoded/trn-on-n24_LUSC_decoded,75.33_fit_time.tsv',
 'train-on-all_decoded/trn-on-n24_MESO_decoded,79.9_fit_time.tsv',
 'train-on-all_decoded/trn-on-n24_OV_dec

In [104]:
len(trn_on_all_pths)

25

In [ ]:
g_devel/train-on-all_F1s

In [106]:
# Train-on-all, exprmt 4, F1 scores

trn_on_all_pths = sorted(glob.glob('train-on-all_decoded/*.tsv'))

re_samp = 5        # Resample at 100%, to train-test splits
trn_tst_splts = 10 # 5

RF_blnk_frm = pd.DataFrame()
DT_blnk_frm = pd.DataFrame()

# for pth in self_decoded_pths: # Toggle here for vae-in / x-decoded
for pth in trn_on_all_pths: # Toggle here for train-on-one_decoded
        
    print(pth)
    df = pd.read_csv(pth, sep="\t", index_col=0)
    c=df.index.name
    # break
    RF_F1_dct = {}
    DT_F1_dct = {}
    
    for j in list(range(0, re_samp)): # 5 resamplings, becomes each bar on plor
        X = df.iloc[:, 1:] # All the samples every loop
        y = df.iloc[:, 0]

        RF_output = [] # to append 5 trn/tst predictions then reset for next resampling
        DT_output = []

        for i in list(range(0, trn_tst_splts)): # 5 train-test splits, from setup
            X_train, X_test, y_train, y_test = train_test_split(
                X, y,
                    test_size=0.33,
                        )
            RF_clf = RandomForestClassifier(max_depth=2) # clf reset
            DT_clf = DecisionTreeClassifier()            

            RF_clf.fit(X_train, y_train)
            DT_clf.fit(X_train, y_train)

            RF_F1_scr = f1_score(y_test, RF_clf.predict(X_test), average='weighted')
            RF_output.append(RF_F1_scr)

            DT_F1_scr = f1_score(y_test, DT_clf.predict(X_test), average='weighted')
            DT_output.append(DT_F1_scr) # end tt loop

        RF_F1_dct['mean'+str(j)] = mean(RF_output) # Take the stats on the tt loop
        RF_F1_dct['std'+str(j)] = stdev(RF_output)

        DT_F1_dct['mean'+str(j)] = mean(DT_output)
        DT_F1_dct['std'+str(j)] = stdev(DT_output) # end sample loop, grown horzontally and ready to stack
    # break

    RF_chrt_frm = pd.DataFrame(RF_F1_dct, index = [c + '_trained_' +

                                             'self' + '_predict_'+                                                   
                                            'rndm_frst_clf'])
    
    # break
    RF_blnk_frm = pd.concat( [RF_blnk_frm, RF_chrt_frm] )
    
    


    DT_chrt_frm = pd.DataFrame(DT_F1_dct, index = [c + '_trained_' +

                                            'self' + '_predict_'+      
                                            'dcsn_tree_clf'])
    DT_blnk_frm = pd.concat( [DT_blnk_frm, DT_chrt_frm] )
    
    
RF_blnk_frm.to_csv(
        'train-on-all_F1s/RF_train-on-all_'+str(trn_tst_splts)+'_trn_tst_splts_F1s.tsv', # Use experiment # var for F1 output dir
        sep = '\t')
DT_blnk_frm.to_csv(
        'train-on-all_F1s/DT_train-on-all_'+str(trn_tst_splts)+'_trn_tst_splts_F1s.tsv', # Use experiment # var for F1 file name, n=24 / self / ..
        sep = '\t')
    
print('all done')

train-on-all_decoded/trn-on-n24_ACC_decoded,75.71_fit_time.tsv
train-on-all_decoded/trn-on-n24_BLCA_decoded,70.71_fit_time.tsv
train-on-all_decoded/trn-on-n24_BRCA_decoded,67.79_fit_time.tsv
train-on-all_decoded/trn-on-n24_CESC_decoded,75.7_fit_time.tsv
train-on-all_decoded/trn-on-n24_COADREAD_decoded,73.74_fit_time.tsv
train-on-all_decoded/trn-on-n24_ESCC_decoded,75.57_fit_time.tsv
train-on-all_decoded/trn-on-n24_GEA_decoded,73.83_fit_time.tsv
train-on-all_decoded/trn-on-n24_HNSC_decoded,74.98_fit_time.tsv
train-on-all_decoded/trn-on-n24_KIRCKICH_decoded,74.78_fit_time.tsv
train-on-all_decoded/trn-on-n24_KIRP_decoded,78.49_fit_time.tsv
train-on-all_decoded/trn-on-n24_LIHCCHOL_decoded,78.19_fit_time.tsv
train-on-all_decoded/trn-on-n24_LUAD_decoded,75.52_fit_time.tsv
train-on-all_decoded/trn-on-n24_LUSC_decoded,75.33_fit_time.tsv
train-on-all_decoded/trn-on-n24_MESO_decoded,79.9_fit_time.tsv
train-on-all_decoded/trn-on-n24_OV_decoded,75.05_fit_time.tsv
train-on-all_decoded/trn-on-n24_PA

#### Archive notes and code, plot series version 16

In [ ]:
for c in clist: # use c for cohort
    print(c)
    # primary_subset = org[org.Labels.str.contains(c)]
    RF_blnk_frm = pd.DataFrame()
    DT_blnk_frm = pd.DataFrame()
    # re_samp = re_samp
    # trn_tst_splts = trn_tst_splts
    for c_path in [s for s in trn_on_one_paths if s.split('/')[1].startswith(c)]:
        print(c_path)
        c_not = c_path.split('_')[3]
        df = pd.read_csv(c_path, sep="\t", index_col=0) ### re-run
        # break
        RF_F1_dct = {} # to a row to stack into the blank frame, refresh at the start of each cancer row
        DT_F1_dct = {}

        # break
        for j in list(range(0, re_samp)): # step through five resamplings
            X = df.iloc[:, 1:] # take all the samples in each of five times through the loop
            y = df.iloc[:, 0]

            RF_output = [] # tack on five tt predictions then reset for next resampling
            DT_output = []

            for i in list(range(0, trn_tst_splts)): # five train-test splits
                X_train, X_test, y_train, y_test = train_test_split(
                    X, y,
                        test_size=0.33,
                            )
                RF_clf = RandomForestClassifier(max_depth=2) # intitialize a fresh clf each tt splt
                DT_clf = DecisionTreeClassifier()            

                RF_clf.fit(X_train, y_train)
                DT_clf.fit(X_train, y_train)

                RF_F1_scr = f1_score(y_test, RF_clf.predict(X_test), average='weighted')
                RF_output.append(RF_F1_scr)

                DT_F1_scr = f1_score(y_test, DT_clf.predict(X_test), average='weighted')
                DT_output.append(DT_F1_scr) # end tt loop

            RF_F1_dct['mean'+str(j)] = mean(RF_output) # Take the stats on the tt loop
            RF_F1_dct['std'+str(j)] = stdev(RF_output)

            DT_F1_dct['mean'+str(j)] = mean(DT_output)
            DT_F1_dct['std'+str(j)] = stdev(DT_output) # end sample loop, grown horzontally and ready to stack

            
        RF_chrt_frm = pd.DataFrame(RF_F1_dct, index = [c + '_trained_' +
                                                 c_not + '_predict_'+
                                                'rndm_frst_clf'])
        RF_blnk_frm = pd.concat( [RF_blnk_frm, RF_chrt_frm] )            

        
        DT_chrt_frm = pd.DataFrame(DT_F1_dct, index = [c + '_trained_' +
                                                 c_not + '_predict_'+
                                                'dcsn_tree_clf'])
        DT_blnk_frm = pd.concat( [DT_blnk_frm, DT_chrt_frm] )
        
        # break # c    
    # break    
    RF_blnk_frm.to_csv(
            'train-on-one_F1s/'+c+'_RF_predict_on_n=24.tsv',
            sep = '\t')
    DT_blnk_frm.to_csv(
            'train-on-one_F1s/'+c+'_DT_predict_on_n=24.tsv',
            sep = '\t')

print('all done') # still no auto-file writing, interactive mode

In [ ]:
# This object is named 'stacked' in the b_step_up_02.ipynb for experiment 2 (step-up)
# Want to rename the train-on-one notebook as such, from trn_one_prdct_anoth series
# Need to split-off the train-on-all

# ^ conclude the three experiments

# Plot version set is 16 for this 2022-11-10 audit

# Try reverse versioning from demo versions
# Outline of target demo versions:
    # -
    
    
    # e_RF_original is template for e_clf_org-dec.ipynb - 

#### Original F1 classical ML RF DT

In [ ]:
org = pd.read_csv('data_GEXP_5k_col_norm/ .tsv', sep="\t", index_col=0)

In [9]:
org.Labels.str.split('_').str[0].unique() # to code snippet templates

array(['ACC', 'BLCA', 'BRCA', 'CESC', 'COADREAD', 'ESCC', 'GEA', 'HNSC',
       'KIRCKICH', 'KIRP', 'LIHCCHOL', 'LUAD', 'LUSC', 'MESO', 'OV',
       'PAAD', 'PCPG', 'PRAD', 'SARC', 'SKCM', 'TGCT', 'THCA', 'THYM',
       'UCEC', 'UVM'], dtype=object)

In [36]:
org.Labels.str.split('_').str[0].unique()[:2]

array(['ACC', 'BLCA'], dtype=object)

In [26]:
# 2022-11-10 audit rebuil, to demo version of classifier e_

RF_blnk_frm = pd.DataFrame()
DT_blnk_frm = pd.DataFrame()

# for c in org.Labels.str.split('_').str[0].unique()[:2]: # use c for cohort
for c in org.Labels.str.split('_').str[0].unique(): # use c for cohort
    print(c)
    primary_subset = org[org.Labels.str.contains(c)]
    
    re_samp = re_samp
    trn_tst_splts = trn_tst_splts
    
    RF_F1_dct = {} # to a row to stack into the blank frame, refresh at the start of each cancer row
    DT_F1_dct = {}
    
    break
    for j in list(range(0, re_samp)): # step through five resamplings
        X = primary_subset.iloc[:, 1:] # take all the samples in each of five times through the loop
        y = primary_subset.iloc[:, 0]
        
        RF_output = [] # tack on five tt predictions then reset for next resampling
        DT_output = []
        
        for i in list(range(0, trn_tst_splts)): # five train-test splits
            X_train, X_test, y_train, y_test = train_test_split(
                X, y,
                    test_size=0.33,
                        )
            RF_clf = RandomForestClassifier(max_depth=2) # intitialize a fresh clf each tt splt
            DT_clf = DecisionTreeClassifier()            
            
            RF_clf.fit(X_train, y_train)
            DT_clf.fit(X_train, y_train)
            
            RF_F1_scr = f1_score(y_test, RF_clf.predict(X_test), average='weighted')
            RF_output.append(RF_F1_scr)
            
            DT_F1_scr = f1_score(y_test, DT_clf.predict(X_test), average='weighted')
            DT_output.append(DT_F1_scr) # end tt loop
                                  
        RF_F1_dct['mean'+str(j)] = mean(RF_output) # Take the stats on the tt loop
        RF_F1_dct['std'+str(j)] = stdev(RF_output)
        
        DT_F1_dct['mean'+str(j)] = mean(DT_output)
        DT_F1_dct['std'+str(j)] = stdev(DT_output) # end sample loop, grown horzontally and ready to stack

    RF_chrt_frm = pd.DataFrame(RF_F1_dct, index = [c + '_' + str(RF_clf)]) # back to cancer loop, make a row DF
    RF_blnk_frm = pd.concat( [RF_blnk_frm, RF_chrt_frm] ) # Put the cancer to the blank storage frame
    
    DT_chrt_frm = pd.DataFrame(DT_F1_dct, index = [c + '_' + str(DT_clf)])
    DT_blnk_frm = pd.concat( [DT_blnk_frm, DT_chrt_frm] )    
        
    # break # c
print('all done') # still no auto-file writing, interactive mode

ACC
all done


In [27]:
primary_subset.tail(1) # this is the decoded F1 generator, was run in d_original, just copy down and clean up retroactively

,Labels,N:GEXP::ALB:213:,N:GEXP::TG:7038:,N:GEXP::COL1A1:1277:,N:GEXP::FN1:2335:,N:GEXP::KRT5:3852:,N:GEXP::KRT14:3861:,N:GEXP::KRT6A:3853:,N:GEXP::ADAM6:8755:,N:GEXP::SERPINA1:5265:,...,N:GEXP::C14orf1:11161:,N:GEXP::SMCHD1:23347:,N:GEXP::ARL2BP:23568:,N:GEXP::PITPNB:23760:,N:GEXP::KDM5A:5927:,N:GEXP::CNIH:10175:,N:GEXP::ACADS:35:,N:GEXP::YIPF2:78992:,N:GEXP::MEPCE:56257:,N:GEXP::PPARG:5468:
TCGA-PK-A5HB,ACC_2,0.000005,0.000006,0.00186,0.004433,0.0,0.0,0.0,0.000008,0.00001,...,0.046216,0.07366,0.241089,0.158638,0.064075,0.207109,0.084056,0.168372,0.242068,0.009073


In [52]:
RF_chrt_frm

,mean0,std0,mean1,std1,mean2,std2,mean3,std3,mean4,std4
UVM_RandomForestClassifier(max_depth=2),0.912437,0.0522,0.909006,0.062281,0.83874,0.157132,0.838622,0.073572,0.932827,0.076454


In [53]:
DT_chrt_frm

,mean0,std0,mean1,std1,mean2,std2,mean3,std3,mean4,std4
UVM_DecisionTreeClassifier(),0.727232,0.116942,0.677946,0.116461,0.765855,0.06267,0.702749,0.105404,0.739251,0.139505


In [43]:
RF_blnk_frm # looks good, pull the breaks, write to file, prep to create and read three decoded versions

,mean0,std0,mean1,std1,mean2,std2,mean3,std3,mean4,std4
ACC_RandomForestClassifier(max_depth=2),0.659503,0.109615,0.575275,0.142284,0.676474,0.141514,0.635788,0.072403,0.605985,0.092675
BLCA_RandomForestClassifier(max_depth=2),0.632672,0.047649,0.567222,0.047515,0.625502,0.090016,0.622351,0.055545,0.581160,0.075009


In [44]:
DT_blnk_frm

,mean0,std0,mean1,std1,mean2,std2,mean3,std3,mean4,std4
ACC_DecisionTreeClassifier(),0.647996,0.111164,0.567689,0.092232,0.530089,0.069986,0.639115,0.100779,0.440670,0.176188
BLCA_DecisionTreeClassifier(),0.695422,0.036382,0.670929,0.016514,0.674870,0.030757,0.679293,0.052988,0.680882,0.020073


In [54]:
RF_blnk_frm.to_csv( # fail
        'original_stacked_F1s/'+'RF'+
        '_org_stacked_MAD_5k_F1s_' +
        str(re_samp)+'_resamps_'+
        str(trn_tst_splts)+'_tt-splits_v'+str(plt_v)+
        '.tsv', sep = '\t')

In [55]:
rf_bfr = pd.read_csv('original_stacked_F1s/'+'RF'+
        '_org_stacked_MAD_5k_F1s_' +
        str(re_samp)+'_resamps_'+
        str(trn_tst_splts)+'_tt-splits_v'+str(plt_v)+
        '.tsv', sep = '\t', index_col = 0) # rando forst blank frame read

In [56]:
rf_bfr.head(1)

,mean0,std0,mean1,std1,mean2,std2,mean3,std3,mean4,std4
ACC_RandomForestClassifier(max_depth=2),0.658843,0.1169,0.628328,0.06689,0.536475,0.185388,0.742396,0.063506,0.721789,0.06964


In [58]:
rf_bfr.tail(2)

,mean0,std0,mean1,std1,mean2,std2,mean3,std3,mean4,std4
UCEC_RandomForestClassifier(max_depth=2),0.552461,0.042559,0.536332,0.052664,0.570853,0.020523,0.555595,0.045932,0.511003,0.080863
UVM_RandomForestClassifier(max_depth=2),0.912437,0.052200,0.909006,0.062281,0.838740,0.157132,0.838622,0.073572,0.932827,0.076454


In [57]:
rf_bfr.shape

(25, 10)

In [49]:
rf_bfr

,mean0,std0,mean1,std1,mean2,std2,mean3,std3,mean4,std4
ACC_RandomForestClassifier(max_depth=2),0.659503,0.109615,0.575275,0.142284,0.676474,0.141514,0.635788,0.072403,0.605985,0.092675
BLCA_RandomForestClassifier(max_depth=2),0.632672,0.047649,0.567222,0.047515,0.625502,0.090016,0.622351,0.055545,0.581160,0.075009


In [59]:
DT_blnk_frm.to_csv( # fail
        'original_stacked_F1s/'+'DT'+
        '_org_stacked_MAD_5k_F1s_' +
        str(re_samp)+'_resamps_'+
        str(trn_tst_splts)+'_tt-splits_v'+str(plt_v)+
        '.tsv', sep = '\t')

In [60]:
dt_bfr = pd.read_csv('original_stacked_F1s/'+'DT'+
        '_org_stacked_MAD_5k_F1s_' +
        str(re_samp)+'_resamps_'+
        str(trn_tst_splts)+'_tt-splits_v'+str(plt_v)+
        '.tsv', sep = '\t', index_col = 0) # rando forst blank frame read

In [61]:
dt_bfr.head(2)

,mean0,std0,mean1,std1,mean2,std2,mean3,std3,mean4,std4
ACC_DecisionTreeClassifier(),0.625967,0.082430,0.592762,0.065542,0.567580,0.045645,0.536205,0.059209,0.600053,0.070981
BLCA_DecisionTreeClassifier(),0.721054,0.039388,0.682029,0.048497,0.661031,0.012896,0.698350,0.020742,0.715822,0.014473


In [62]:
dt_bfr.tail(2)

,mean0,std0,mean1,std1,mean2,std2,mean3,std3,mean4,std4
UCEC_DecisionTreeClassifier(),0.529796,0.031473,0.517575,0.026576,0.537442,0.027094,0.578019,0.063398,0.534677,0.049953
UVM_DecisionTreeClassifier(),0.727232,0.116942,0.677946,0.116461,0.765855,0.062670,0.702749,0.105404,0.739251,0.139505


In [63]:
dt_bfr.shape

(25, 10)

In [ ]:
# End RF and DT on original

#### Train-on-one F1 production for d_train-on-one experiment

Target dir for F1s: 

In [8]:
mkdir train-on-one_F1s

In [4]:
trn_on_one_dec = pd.read_csv('train-on-one_decoded/ACC_trained_BLCA_decoded_3.35_fit_time.tsv', sep="\t", index_col=0)

In [29]:
trn_on_one_dec

,Labels,N:GEXP::ALB:213:,N:GEXP::TG:7038:,N:GEXP::COL1A1:1277:,N:GEXP::FN1:2335:,N:GEXP::KRT5:3852:,N:GEXP::KRT14:3861:,N:GEXP::KRT6A:3853:,N:GEXP::ADAM6:8755:,N:GEXP::SERPINA1:5265:,...,N:GEXP::C14orf1:11161:,N:GEXP::SMCHD1:23347:,N:GEXP::ARL2BP:23568:,N:GEXP::PITPNB:23760:,N:GEXP::KDM5A:5927:,N:GEXP::CNIH:10175:,N:GEXP::ACADS:35:,N:GEXP::YIPF2:78992:,N:GEXP::MEPCE:56257:,N:GEXP::PPARG:5468:
TCGA-2F-A9KO,BLCA_2,0.266226,0.294665,0.287804,0.287290,0.299087,0.262060,0.260004,0.270740,0.284690,...,0.371561,0.321058,0.462665,0.348813,0.347686,0.342063,0.333076,0.373408,0.405937,0.276956
TCGA-2F-A9KP,BLCA_2,0.285378,0.285905,0.283878,0.297263,0.296055,0.277293,0.270595,0.276574,0.266651,...,0.406567,0.321803,0.461659,0.347546,0.349164,0.362311,0.332299,0.363754,0.402099,0.280254
TCGA-2F-A9KQ,BLCA_2,0.329344,0.357496,0.338200,0.336121,0.350700,0.336602,0.339676,0.324306,0.324085,...,0.405215,0.360598,0.476952,0.387721,0.389815,0.392136,0.395932,0.413052,0.430038,0.344641
TCGA-2F-A9KR,BLCA_1,0.271298,0.312547,0.283140,0.283972,0.306987,0.280454,0.274627,0.272107,0.279609,...,0.376178,0.331563,0.469330,0.365567,0.353976,0.358204,0.337693,0.380484,0.411087,0.272624
TCGA-2F-A9KT,BLCA_1,0.307971,0.322204,0.302893,0.322415,0.311992,0.308697,0.296239,0.307042,0.303178,...,0.416961,0.342644,0.466223,0.369281,0.372384,0.369681,0.347804,0.390589,0.421292,0.298631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-ZF-AA56,BLCA_1,0.292657,0.317755,0.299830,0.313446,0.314278,0.307970,0.286017,0.298050,0.300637,...,0.407025,0.341444,0.461919,0.361329,0.381027,0.387305,0.352206,0.393642,0.422552,0.299662
TCGA-ZF-AA58,BLCA_1,0.234231,0.258396,0.246750,0.256307,0.259360,0.251576,0.234416,0.238788,0.240425,...,0.363516,0.296872,0.455211,0.318696,0.337239,0.338449,0.310399,0.346221,0.390676,0.235789
TCGA-ZF-AA5H,BLCA_1,0.221841,0.250364,0.238526,0.261122,0.251615,0.247287,0.231672,0.240933,0.236034,...,0.362792,0.295693,0.463476,0.309523,0.342166,0.329351,0.299656,0.335697,0.396835,0.227859
TCGA-ZF-AA5N,BLCA_2,0.317559,0.325121,0.326246,0.334373,0.325583,0.345065,0.316175,0.319461,0.314304,...,0.428102,0.348623,0.471264,0.370316,0.379281,0.396871,0.374153,0.399158,0.434218,0.323460


In [ ]:
trn_on_one_dec

In [11]:
import glob

In [13]:
len(sorted(glob.glob('train-on-one_decoded/*.tsv')))

600

In [14]:
trn_on_one_paths = sorted(glob.glob('train-on-one_decoded/*.tsv'))

In [15]:
trn_on_one_paths

['train-on-one_decoded/ACC_trained_BLCA_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_BRCA_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_CESC_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_COADREAD_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_ESCC_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_GEA_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_HNSC_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_KIRCKICH_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_KIRP_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_LIHCCHOL_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_LUAD_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_LUSC_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_MESO_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_OV_decoded_3.35_fit_time.tsv',
 'train-on-one_decoded/ACC_trained_PAAD

In [16]:
trn_on_one_paths[0]

'train-on-one_decoded/ACC_trained_BLCA_decoded_3.35_fit_time.tsv'

In [19]:
path.split('/')[1].split('_')[0]

'ACC'

In [21]:
cohorts = []
for path in trn_on_one_paths:
    # print(path)
    # c=path.split
    cohorts.append(path.split('/')[1].split('_')[0])
    # break

In [23]:
cohorts = set(cohorts)

In [24]:
cohorts

{'ACC',
 'BLCA',
 'BRCA',
 'CESC',
 'COADREAD',
 'ESCC',
 'GEA',
 'HNSC',
 'KIRCKICH',
 'KIRP',
 'LIHCCHOL',
 'LUAD',
 'LUSC',
 'MESO',
 'OV',
 'PAAD',
 'PCPG',
 'PRAD',
 'SARC',
 'SKCM',
 'TGCT',
 'THCA',
 'THYM',
 'UCEC',
 'UVM'}

In [47]:
list(cohorts).sort()

In [49]:
clist = list(cohorts)

In [50]:
clist.sort()

In [51]:
clist

['ACC',
 'BLCA',
 'BRCA',
 'CESC',
 'COADREAD',
 'ESCC',
 'GEA',
 'HNSC',
 'KIRCKICH',
 'KIRP',
 'LIHCCHOL',
 'LUAD',
 'LUSC',
 'MESO',
 'OV',
 'PAAD',
 'PCPG',
 'PRAD',
 'SARC',
 'SKCM',
 'TGCT',
 'THCA',
 'THYM',
 'UCEC',
 'UVM']

In [ ]:
# referring to b_model_trn_one_prdct_anoth_01.ipynb

In [30]:
%whos

Variable                 Type         Data/Info
-----------------------------------------------
DT_F1_dct                dict         n=0
DT_blnk_frm              DataFrame    Empty DataFrame\nColumns: []\nIndex: []
DecisionTreeClassifier   ABCMeta      <class 'sklearn.tree._cla<...>.DecisionTreeClassifier'>
RF_F1_dct                dict         n=0
RF_blnk_frm              DataFrame    Empty DataFrame\nColumns: []\nIndex: []
RandomForestClassifier   ABCMeta      <class 'sklearn.ensemble.<...>.RandomForestClassifier'>
c                        str          ACC
cohorts                  set          {'LUAD', 'TGCT', 'KIRP', <...>', 'ESCC', 'ACC', 'BRCA'}
f1_score                 function     <function f1_score at 0x147dfa310>
glob                     module       <module 'glob' from '/Use<...>9/lib/python3.9/glob.py'>
mean                     function     <function mean at 0x179bf3dc0>
org                      DataFrame                 Labels  N:GE<...>8009 rows x 5001 columns]
path      

In [31]:
plt_v

16

In [32]:
trn_tst_splts

5

In [33]:
re_samp

5

In [35]:
c

'ACC'

In [36]:
c = 'BLCA'

In [38]:
for c_path in [s for s in trn_on_one_paths if s.split('/')[1].startswith(c)]:
    print(c_path)
    # break

train-on-one_decoded/BLCA_trained_ACC_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_BRCA_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_CESC_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_COADREAD_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_ESCC_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_GEA_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_HNSC_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_KIRCKICH_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_KIRP_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_LIHCCHOL_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_LUAD_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_LUSC_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_MESO_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_OV_decoded_11.32_fit_time.tsv
train-on-one_decoded/BLCA_trained_PAAD_decoded_11.32_fit_time.tsv
tr

In [ ]:
for chrt_path in [s for s in t1pa_paths if s.split('/')[1].startswith(chrt)]:

In [52]:
for c in clist:
    print(c)
    for c_path in [s for s in trn_on_one_paths if s.split('/')[1].startswith(c)]:
        print(c_path)
    break

ACC
train-on-one_decoded/ACC_trained_BLCA_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_BRCA_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_CESC_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_COADREAD_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_ESCC_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_GEA_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_HNSC_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_KIRCKICH_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_KIRP_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_LIHCCHOL_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_LUAD_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_LUSC_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_MESO_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_OV_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_PAAD_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_tr

In [ ]:
# 2022-11-11, Friday, generate F1s for input to d_train-on-one plota
# RF_blnk_frm = pd.DataFrame()
# DT_blnk_frm = pd.DataFrame()

# for c in org.Labels.str.split('_').str[0].unique()[:2]: # use c for cohort

for c in clist: # use c for cohort
    print(c)
    # primary_subset = org[org.Labels.str.contains(c)]
    RF_blnk_frm = pd.DataFrame()
    DT_blnk_frm = pd.DataFrame()
    # re_samp = re_samp
    # trn_tst_splts = trn_tst_splts
    for c_path in [s for s in trn_on_one_paths if s.split('/')[1].startswith(c)]:
        print(c_path)
        c_not = c_path.split('_')[3]
        df = pd.read_csv(c_path, sep="\t", index_col=0) ### re-run
        # break
        RF_F1_dct = {} # to a row to stack into the blank frame, refresh at the start of each cancer row
        DT_F1_dct = {}

        # break
        for j in list(range(0, re_samp)): # step through five resamplings
            X = df.iloc[:, 1:] # take all the samples in each of five times through the loop
            y = df.iloc[:, 0]

            RF_output = [] # tack on five tt predictions then reset for next resampling
            DT_output = []

            for i in list(range(0, trn_tst_splts)): # five train-test splits
                X_train, X_test, y_train, y_test = train_test_split(
                    X, y,
                        test_size=0.33,
                            )
                RF_clf = RandomForestClassifier(max_depth=2) # intitialize a fresh clf each tt splt
                DT_clf = DecisionTreeClassifier()            

                RF_clf.fit(X_train, y_train)
                DT_clf.fit(X_train, y_train)

                RF_F1_scr = f1_score(y_test, RF_clf.predict(X_test), average='weighted')
                RF_output.append(RF_F1_scr)

                DT_F1_scr = f1_score(y_test, DT_clf.predict(X_test), average='weighted')
                DT_output.append(DT_F1_scr) # end tt loop

            RF_F1_dct['mean'+str(j)] = mean(RF_output) # Take the stats on the tt loop
            RF_F1_dct['std'+str(j)] = stdev(RF_output)

            DT_F1_dct['mean'+str(j)] = mean(DT_output)
            DT_F1_dct['std'+str(j)] = stdev(DT_output) # end sample loop, grown horzontally and ready to stack

            
        RF_chrt_frm = pd.DataFrame(RF_F1_dct, index = [c + '_trained_' +
                                                 c_not + '_predict_'+
                                                'rndm_frst_clf'])
        RF_blnk_frm = pd.concat( [RF_blnk_frm, RF_chrt_frm] )            

        
        DT_chrt_frm = pd.DataFrame(DT_F1_dct, index = [c + '_trained_' +
                                                 c_not + '_predict_'+
                                                'dcsn_tree_clf'])
        DT_blnk_frm = pd.concat( [DT_blnk_frm, DT_chrt_frm] )
        
        # break # c    
    # break    
    RF_blnk_frm.to_csv(
            'train-on-one_F1s/'+c+'_RF_predict_on_n=24.tsv',
            sep = '\t')
    DT_blnk_frm.to_csv(
            'train-on-one_F1s/'+c+'_DT_predict_on_n=24.tsv',
            sep = '\t')

print('all done') # still no auto-file writing, interactive mode

ACC
train-on-one_decoded/ACC_trained_BLCA_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_BRCA_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_CESC_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_COADREAD_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_ESCC_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_GEA_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_HNSC_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_KIRCKICH_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_KIRP_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_LIHCCHOL_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_LUAD_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_LUSC_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_MESO_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_OV_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_trained_PAAD_decoded_3.35_fit_time.tsv
train-on-one_decoded/ACC_tr

In [ ]:
# Templates / notes - delete for demo version

In [ ]:
for clf_name in ['RF', 'DT']:
    blnk_frm.to_csv( # fail
        'original_stacked_F1s/'+clf_name+
        '_org_stacked_MAD_5k_F1s_' +
        str(re_samp)+'_resamps_'+
        str(trn_tst_splts)+'_tt-splits_v'+str(plt_v)+
        '.tsv', sep = '\t')

In [ ]:
bfr = pd.read_csv('original_stacked_F1s/5k_r13.tsv', sep = '\t', index_col = 0) # blank frame read

In [ ]:
# from b_model_trn_one_prdct_anoth_01, use to build a 5k stacked original version above:

# DELETE IN DEMO VERSION

In [ ]:
 # Original baseline F1 scores on 5k feats for plotting - v5, redoing on M1 machine
plt_v = 13         # to plot naming, 13 is first M1 version

re_samp = 5 # v13, still running at 100%
trn_tst_splts = 5 # trn_tst_splts   

blnk_frm = pd.DataFrame()

for path in paths:
    
    cohort = path.split('/')[1].split('.')[0] # should still work
    print(cohort)
    # break # check
    df = pd.read_csv(path, sep="\t", index_col=0)
    features = df.iloc[:, 1:].mad().sort_values(ascending=False)[:5000].index
    normVals = df[features].max()
    vals = df[features]
    X_5k = (vals / normVals)
    df = pd.concat( [df.Labels, X_5k] , axis = 1) # Overwrite df with 5k mad feats
    # break
    F1_dct = {}
    for j in list(range(0, re_samp)): # 100% sampling rate
        X = df.iloc[:, 1:]
        y = df.iloc[:, 0]
        output = []
        for i in list(range(0, trn_tst_splts)): # number of train test splits
            X_train, X_test, y_train, y_test = train_test_split(
                X, y,
                    test_size=0.33,
                        )
            clf = RandomForestClassifier(max_depth=2, # reset the classifier each time through loop
                )
            clf.fit(X_train, y_train)
            F1_scr = f1_score(y_test, clf.predict(X_test), average='weighted')
            output.append(F1_scr) # extracting stats and overwriting, str(clf)
                                  
        F1_dct['mean'+str(j)] = mean(output)
        F1_dct['std'+str(j)] = stdev(output)

    chrt_frm = pd.DataFrame(F1_dct, index = [cohort + '_' + str(clf)])
    blnk_frm = pd.concat( [blnk_frm, chrt_frm] )
    # print('path loop done')

print('all done') # no auto-file writing, interactive mode